In [1]:
import json
import os
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import scipy.sparse as sp
from sklearn.linear_model import LogisticRegression
random.seed(42)
np.random.seed(42)

In [2]:
pd.set_option('display.max_column', None)

In [3]:
OUTPUT_DATA_DIR = "../output_data/"

train_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_training.csv")
val_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_validation.csv")

train_df_processed['cleaned_text'] = train_df_processed['cleaned_text'].apply(lambda x: "" if pd.isnull(x) else x)
val_df_processed['cleaned_text'] = val_df_processed['cleaned_text'].apply(lambda x: "" if pd.isnull(x) else x)

/home/farid/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
columns_to_keep = ['text_reviews_count', 'is_ebook', 'average_rating', 'num_pages',
                   'ratings_count', 'is_translated', 'is_in_series',
                   'series_length', 'is_paperback', 'is_hardcover', 'is_audio', 'is_other_format',
                   'from_penguin', 'from_harpercollins', 'from_university_press', 'from_vintage',
                   'from_createspace', 'other_publisher', 'author_a', 'author_b', 'author_c',
                   'author_d', 'author_e', 'author_f', 'author_other']
X_train_reg = train_df_processed[columns_to_keep]
X_val_reg = val_df_processed[columns_to_keep]

In [5]:
def log_transform_columns(data_df, cols):
    """Applies a log transform to `cols` in `data_df`.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame in which the columns will be transformed.
    cols: collection
        The columns in `data_df` to be log scaled.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after log scaling
        the columns `cols`.

    """
    for col in cols:
        data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)
    return data_df

In [6]:
log_transform_cols = ['text_reviews_count', 'ratings_count']
X_train_reg = log_transform_columns(X_train_reg, log_transform_cols)
X_val_reg = log_transform_columns(X_val_reg, log_transform_cols)

<ipython-input-5-49ee2668773d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)


In [7]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_train_reg = min_max_scaler.fit_transform(X_train_reg)
X_val_reg = min_max_scaler.transform(X_val_reg)

In [8]:
train_df_processed.sample(3)

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,ratings_count,work_id,title,title_without_series,shelved,read,rated,recommended,year_month_added,year_month_updated,pub_date,is_translated,main_author,is_in_series,series_length,title_description,read_count,rated_count,recommended_count,shelved_count,is_paperback,is_hardcover,is_audio,is_other_format,from_penguin,from_harpercollins,from_university_press,from_vintage,from_createspace,other_publisher,author_a,author_b,author_c,author_d,author_e,author_f,author_other,cleaned_text
118172,16b09d050bdd8d9c23920b811f51472d,2884170,cc454745d4506b7ab880a8561b5d2ac6,False,0,NaN,Wed Feb 06 19:21:46 -0800 2013,Wed Feb 06 19:21:46 -0800 2013,NaN,NaN,0374317763,389,[],US,eng,"[{'count': '1536', 'name': 'to-read'}, {'count...",NaN,False,3.97,B01IN8ZVUO,"['1626753', '2149610', '4296443', '142097', '1...",There's\nmore to me than\nmost people\nsee.\nT...,hardcover,https://www.goodreads.com/book/show/2884170-di...,"[{'author_id': '49193', 'role': ''}]","farrar, straus and giroux (byr)",128.0,1.0,9780374317768,4.0,NaN,2008.0,https://www.goodreads.com/book/show/2884170-di...,https://images.gr-assets.com/books/1317064870m...,2489,2910398,Diamond Willow,Diamond Willow,1,0,0,0,2013-02,2013-02,2008-04,0,49193.0,0,1,Diamond Willow There's\nmore to me than\nmost ...,3,3,1,84,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,diamond willow peopl see twelv year old willow...
91341,145962ddd9ce5921b943ec2f8024dc66,427278,0ea83fb8278cfd7240fcf6486f1d3947,True,4,NaN,Sat Jul 30 10:12:00 -0700 2016,Tue Sep 06 06:27:15 -0700 2016,Tue Sep 06 06:27:15 -0700 2016,Sat Jul 30 10:12:01 -0700 2016,014243731X,35,[],US,eng,"[{'count': '5673', 'name': 'to-read'}, {'count...",NaN,False,4.23,B002ZZQD3Q,"['75504', '65349', '134025', '97553', '65336',...",This volume includes the title poem as well as...,paperback,https://www.goodreads.com/book/show/427278.The...,"[{'author_id': '18540', 'role': ''}, {'author_...",penguin classics,108.0,25.0,9780142437315,2.0,NaN,2003.0,https://www.goodreads.com/book/show/427278.The...,https://images.gr-assets.com/books/1309376891m...,467,15306654,The Waste Land and Other Poems,The Waste Land and Other Poems,1,1,1,1,2016-07,2016-09,2003-02,0,18540.0,0,1,The Waste Land and Other Poems This volume inc...,3,3,3,7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,wast land poem volum includ titl poem well lov...
118606,e96cd537c92178024db62bced19f42cd,2696,7ee85335002eea2abf9f97d475c13294,True,3,NaN,Sun Sep 07 10:25:07 -0700 2014,Sun Sep 07 10:25:07 -0700 2014,NaN,NaN,0140424385,2071,[],US,eng,"[{'count': '3097', 'name': 'to-read'}, {'count...",NaN,False,3.48,B018HCIHXY,"['3049', '449589', '119079', '51799', '765427'...",The procession that crosses Chaucer's pages is...,paperback,https://www.goodreads.com/book/show/2696.The_C...,"[{'author_id': '1838', 'role': ''}, {'author_i...",penguin classics,504.0,30.0,9780140424386,1.0,NaN,2003.0,https://www.goodreads.com/book/show/2696.The_C...,https://images.gr-assets.com/books/1261208589m...,150099,986234,The Canterbury Tales,The Canterbury Tales,1,1,1,0,2014-09,2014-09,2003-01,0,1838.0,0,1,The Canterbury Tales The procession that cross...,2,2,1,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,canterburi tale process cross chaucer page ful...


In [9]:
all_df = pd.concat([train_df_processed, val_df_processed])
book_df = all_df[['book_id', 'cleaned_text']]
book_df = book_df.drop_duplicates(subset=['book_id'])
book_df['cleaned_text'] = book_df['cleaned_text'].apply(lambda x: "" if pd.isnull(x) else x)

In [10]:
book_df.sample(3)

,book_id,cleaned_text
44623,23887164,anti british lefti commi traitor scum took fla...
52634,20738843,heroid altern cover edit isbn13 9780140423556 ...
981,782095,midnight ride paul rever longfellow tribut fam...


In [11]:
w2v = Word2Vec(list(book_df['cleaned_text']), size=200, window=10, min_count=1)

In [12]:
def create_book_vector(book_text, vec_length):
    """Creates a vector for the book given by `book_text`.

    The word vectors for each word in `book_text` are
    averaged to build a vector for the book.

    Parameters
    ----------
    book_text: str
        The book text for which the vector is generated.

    Returns
    -------
    vector
        A vector for the book.

    """
    text_vecs = [word for word in book_text if word in w2v.wv.vocab]
    if len(text_vecs) > 0:
        return np.mean(w2v[text_vecs], axis=0)
    return np.zeros(vec_length)

In [13]:
book_df['book_vector'] = book_df['cleaned_text'].apply(lambda x: create_book_vector(x, 200))


<ipython-input-12-9ef269e54925>:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[text_vecs], axis=0)


In [20]:
train_df_processed = train_df_processed.merge(book_df[['book_id','book_vector']], on='book_id', how='left')
val_df_processed = val_df_processed.merge(book_df[['book_id','book_vector']], on='book_id', how='left')

In [21]:
train_df_processed.sample(3)

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,ratings_count,work_id,title,title_without_series,shelved,read,rated,recommended,year_month_added,year_month_updated,pub_date,is_translated,main_author,is_in_series,series_length,title_description,read_count,rated_count,recommended_count,shelved_count,is_paperback,is_hardcover,is_audio,is_other_format,from_penguin,from_harpercollins,from_university_press,from_vintage,from_createspace,other_publisher,author_a,author_b,author_c,author_d,author_e,author_f,author_other,cleaned_text,book_vector
253940,da372b1a8cede63a6517f1f7266d443b,23919,9ed1829198bdd66a021d96298b5f9d8f,True,5,NaN,Sat Jan 12 16:24:21 -0800 2013,Sat Jan 12 16:24:21 -0800 2013,NaN,NaN,0385074077,1094,[],US,eng,"[{'count': '1743', 'name': 'to-read'}, {'count...",NaN,False,4.37,B076B1QYXK,"['5289', '569564', '8320840', '95819', '112204...",The life of American writer Edgar Allan Poe wa...,NaN,https://www.goodreads.com/book/show/23919.The_...,"[{'author_id': '4624490', 'role': ''}]","doubleday & company, inc.",821.0,15.0,9780385074070,8.0,NaN,1984.0,https://www.goodreads.com/book/show/23919.The_...,https://images.gr-assets.com/books/1327942676m...,161550,30431,The Complete Stories and Poems,The Complete Stories and Poems,1,1,1,1,2013-01,2013-01,1984-08,0,4624490.0,0,1,The Complete Stories and Poems The life of Ame...,1,1,1,4,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,complet stori poem life american writer edgar ...,"[0.106511086, 0.021732124, -0.027596388, -0.10..."
87547,f547e8686f9f0e7d02040e25318e2876,1420,671a90a165e577998b73d275ff3dddb3,True,4,NaN,Sat May 10 18:44:02 -0700 2014,Sat May 10 18:44:02 -0700 2014,NaN,NaN,0521618746,5737,[],US,eng,"[{'count': '1659', 'name': 'school'}, {'count'...",NaN,False,4.01,B008TVM1JU,"['383337', '92250', '52823', '18545', '12287',...","One of the greatest plays of all time, the com...",paperback,https://www.goodreads.com/book/show/1420.Hamlet,"[{'author_id': '947', 'role': ''}, {'author_id...",cambridge university press,289.0,1.0,9780521618748,8.0,NaN,2005.0,https://www.goodreads.com/book/show/1420.Hamlet,https://images.gr-assets.com/books/1351051208m...,526122,1885548,Hamlet,Hamlet,1,1,1,1,2014-05,2014-05,2005-08,0,947.0,0,1,"Hamlet One of the greatest plays of all time, ...",2,1,0,11,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,hamlet one greatest play time compel tragedi t...,"[0.099180356, 0.014878449, -0.013121638, -0.08..."
96802,d3dc44041e0af824ac398d23f177b12c,21808019,1f34f401e1d6d7fd51adc2270ae42194,False,0,NaN,Tue May 30 08:41:34 -0700 2017,Tue May 30 08:41:34 -0700 2017,NaN,NaN,1304856550,23,[],US,eng,"[{'count': '1443', 'name': 'to-read'}, {'count...",NaN,False,4.02,B01EOIWI6M,"['722291', '19572070', '20557173', '449770', '...",Robert M. Drake brings you a collection of quo...,paperback,https://www.goodreads.com/book/show/21808019-s...,"[{'author_id': '8445480', 'role': ''}]",lulu.com,170.0,25.0,9781304856555,8.0,NaN,2015.0,https://www.goodreads.com/book/show/21808019-s...,https://images.gr-assets.com/books/1399172019m...,305,41075142,Spaceship: A Collection of Words for the Misun...,Spaceship: A Collection of Words for the Misun...,1,0,0,0,2017-05,2017-05,2015-08,0,8445480.0,0,1,Spaceship: A Collection of Words for the Misun...,7,7,6,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,spaceship collect word misunderstood robert dr...,"[0.119358584, 0.021023327, -0.001833487, -0.10..."


In [22]:
val_df_processed.sample(3)

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,ratings_count,work_id,title,title_without_series,shelved,read,rated,recommended,year_month_added,year_month_updated,pub_date,is_translated,main_author,is_in_series,series_length,title_description,read_count,rated_count,recommended_count,shelved_count,is_paperback,is_hardcover,is_audio,is_other_format,from_penguin,from_harpercollins,from_university_press,from_vintage,from_createspace,other_publisher,author_a,author_b,author_c,author_d,author_e,author_f,author_other,cleaned_text,book_vector
22848,e45027919c17733c065d81ac07a70389,6295,c5496174daf78720438f0f6845b3feb1,True,5,NaN,Wed Jan 16 19:16:50 -0800 2013,Wed Jan 16 19:16:50 -0800 2013,NaN,NaN,0872863107,1104,[],US,eng,"[{'count': '15885', 'name': 'to-read'}, {'coun...",NaN,False,4.14,NaN,"['99713', '139867', '534647', '57903', '65336'...",The prophetic poem that launched a generation ...,hardcover,https://www.goodreads.com/book/show/6295.Howl_...,"[{'author_id': '4261', 'role': ''}, {'author_i...",city lights,56.0,1.0,9780872863101,1.0,NaN,2001.0,https://www.goodreads.com/book/show/6295.Howl_...,https://images.gr-assets.com/books/1327870926m...,73977,2290688,Howl and Other Poems,Howl and Other Poems,1,1,1,1,2013-01,2013-01,2001-01,0,4261.0,0,1,Howl and Other Poems The prophetic poem that l...,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,howl poem prophet poem launch gener first publ...,"[0.106938966, 0.01585419, -0.025656892, -0.099..."
2250,660a9a2a2c427d8ab981ae5457b192b3,46231,90e1b7b69b6c6b4800fc1c1f00582fa3,True,4,NaN,Fri Apr 30 10:46:33 -0700 2010,Fri Apr 30 10:47:58 -0700 2010,Fri Apr 30 10:47:58 -0700 2010,Fri Apr 30 00:00:00 -0700 2010,0140150749,304,[],US,en-US,"[{'count': '5339', 'name': 'to-read'}, {'count...",NaN,False,4.34,NaN,"['181120', '108128', '165251', '199157', '6975...","This collection ranges over the verse, stories...",paperback,https://www.goodreads.com/book/show/46231.The_...,"[{'author_id': '24956', 'role': ''}, {'author_...",penguin books,610.0,9.0,9.78014e+12,12.0,The Viking Portable Library,1976.0,https://www.goodreads.com/book/show/46231.The_...,https://images.gr-assets.com/books/1386926061m...,8776,43801,The Portable Dorothy Parker,The Portable Dorothy Parker,1,1,1,1,2010-04,2010-04,1976-12,0,24956.0,0,1,The Portable Dorothy Parker This collection ra...,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,portabl dorothi parker collect rang vers stori...,"[0.12002213, 0.026860293, -0.0020013698, -0.06..."
7900,261616ad71af5a8e49d21c7f7605a5eb,91571,e99527af4c83989f0e07f14f07f71d93,False,0,NaN,Tue Jul 12 20:48:54 -0700 2016,Tue Jul 12 20:48:55 -0700 2016,NaN,NaN,0743484908,461,[],US,eng,"[{'count': '1093', 'name': 'to-read'}, {'count...",NaN,False,3.67,B00YNIIRLY,"['613947', '2048536', '295771', '143487', '383...",Measure for Measureis among the most passionat...,paperback,https://www.goodreads.com/book/show/91571.Meas...,"[{'author_id': '947', 'role': ''}, {'author_id...",NaN,208.0,NaN,9.78074e+12,NaN,Folger Shakespeare Library,NaN,https://www.goodreads.com/book/show/91571.Meas...,https://s.gr-assets.com/assets/nophoto/book/11...,15777,278178,Measure for Measure,Measure for Measure,1,0,0,0,2016-07,2016-07,-00,0,947.0,0,1,Measure for Measure Measure for Measureis amon...,4,4,2,7,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,measur measur measur measurei among passion di...,"[0.11116948, 0.01675823, -0.019437926, -0.1001..."


In [23]:
def create_book_vec_df(book_vecs, indices):
    """Creates a dataframe from `book_vecs`.

    Each numpy array in `book_vecs` is converted to a
    row in the resulting dataframe.

    Parameters
    ----------
    book_vecs: list
        A list of numpy arrays where each array corresponds
        to the book vector for a book.
    indicies: np.array
        A numpy array of indices for the DataFrame

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from converting `review_vecs`
        to a dataframe.

    """
    book_vec_df = pd.DataFrame(np.vstack(book_vecs))
    book_vec_df.columns = ["word" + str(col) for col in book_vec_df.columns]
    book_vec_df.index = indices
    return book_vec_df

In [24]:
train_wv = create_book_vec_df(train_df_processed['book_vector'], train_df_processed.index)
val_wv = create_book_vec_df(val_df_processed['book_vector'], val_df_processed.index)

In [25]:
train_wv.sample(5)

,word0,word1,word2,word3,word4,word5,word6,word7,word8,word9,word10,word11,word12,word13,word14,word15,word16,word17,word18,word19,word20,word21,word22,word23,word24,word25,word26,word27,word28,word29,word30,word31,word32,word33,word34,word35,word36,word37,word38,word39,word40,word41,word42,word43,word44,word45,word46,word47,word48,word49,word50,word51,word52,word53,word54,word55,word56,word57,word58,word59,word60,word61,word62,word63,word64,word65,word66,word67,word68,word69,word70,word71,word72,word73,word74,word75,word76,word77,word78,word79,word80,word81,word82,word83,word84,word85,word86,word87,word88,word89,word90,word91,word92,word93,word94,word95,word96,word97,word98,word99,word100,word101,word102,word103,word104,word105,word106,word107,word108,word109,word110,word111,word112,word113,word114,word115,word116,word117,word118,word119,word120,word121,word122,word123,word124,word125,word126,word127,word128,word129,word130,word131,word132,word133,word134,word135,word136,word137,word138,word139,word140,word141,word142,word143,word144,word145,word146,word147,word148,word149,word150,word151,word152,word153,word154,word155,word156,word157,word158,word159,word160,word161,word162,word163,word164,word165,word166,word167,word168,word169,word170,word171,word172,word173,word174,word175,word176,word177,word178,word179,word180,word181,word182,word183,word184,word185,word186,word187,word188,word189,word190,word191,word192,word193,word194,word195,word196,word197,word198,word199
28574,0.109503,0.018608,-0.024836,-0.095200,0.004889,-0.000769,-0.085638,-0.149407,0.105933,-0.022740,-0.145364,0.009297,-0.081702,0.112135,0.173023,-0.101105,0.077007,-0.074143,-0.079057,0.029028,-0.010770,-0.076842,-0.036661,-0.020126,0.094144,-0.063885,0.088389,-0.098815,0.021707,-0.023335,0.038872,0.046109,-0.011143,0.011769,0.089949,0.027322,0.001778,0.035284,-0.070337,0.014828,0.031878,0.037682,0.026238,-0.038262,-0.083156,-0.109643,-0.078642,-0.009737,0.017116,-0.025432,0.078706,0.056716,0.027521,0.031540,0.056533,0.123482,-0.076134,0.037045,0.081468,-0.061425,0.033743,-0.041628,0.094657,0.053279,-0.033254,-0.141680,0.017471,-0.010584,0.036239,0.012616,-0.092293,0.016083,0.060939,0.070471,-0.051060,0.098063,-0.011857,0.005222,-0.055276,-0.013247,0.016136,-0.000361,0.063320,0.004118,0.027813,0.008784,-0.005991,0.050855,0.068310,0.024535,0.043084,0.089531,0.042692,-0.047256,0.039482,-0.079194,-0.014663,-0.036187,-0.006814,0.036710,-0.069462,0.025286,0.126476,-0.041659,0.074090,0.027115,0.013086,0.011989,-0.031197,-0.035179,-0.016818,-0.005702,0.058052,0.047567,0.008624,-0.019127,0.023200,0.005752,0.023220,-0.013508,-0.019675,-0.002784,-0.026877,0.039282,-0.013582,-0.048741,-0.040612,-0.011827,-0.016876,-0.133512,-0.003787,0.050688,-0.032127,-0.037875,0.101663,-0.019262,0.003755,-0.022394,0.034508,0.041569,0.037059,0.007508,0.013437,0.067507,-0.069706,0.048649,-0.185037,0.025721,0.038282,-0.002893,-0.055432,0.034242,-0.044456,0.071645,0.025007,-0.109484,-0.032936,0.007626,-0.020102,-0.066261,-0.007187,-0.015475,0.053738,-0.011996,-0.032073,0.023363,-0.017774,0.038476,0.004779,0.023117,0.011752,0.021947,0.028845,0.051272,0.027257,-0.036734,-0.062294,-0.008339,-0.091925,0.044766,-0.022696,0.054285,0.036358,0.007429,0.013237,0.029952,-0.055229,0.086044,-0.030331,-0.048145,0.000441,-0.030695,0.001601,-0.001371,0.048880,0.022258,-0.040414,0.008732,0.005474,0.049266
139879,0.131225,0.027926,-0.007823,-0.096329,-0.012175,-0.012695,-0.098908,-0.164642,0.118825,-0.019400,-0.173514,0.002755,-0.081331,0.129734,0.190446,-0.096229,0.088426,-0.078365,-0.077922,0.021984,-0.006724,-0.083170,-0.043096,-0.030283,0.115759,-0.077534,0.101136,-0.124774,0.035693,-0.044135,0.042861,0.056305,-0.016415,0.006987,0.093751,0.022646,0.002974,0.036050,-0.075056,0.023369,0.025926,0.027227,0.021662,-0.028341,-0.087596,-0.109972,-0.085573,-0.020280,0.033624,-0.022821,0.090280,0.067584,0.031813,0.024796,0.058584,0.133077,-0.074373,0.031550,0.094915,-0.065345,0.023998,-0.050724,0.111490,0.052000

In [26]:
X_train_reg_df = pd.DataFrame(np.vstack(X_train_reg))
X_train_reg_df.index = train_df_processed.index

X_val_reg_df = pd.DataFrame(np.vstack(X_val_reg))
X_val_reg_df.index = val_df_processed.index

X_train_wv_reg = sp.csr_matrix(pd.concat([train_wv, X_train_reg_df], axis=1))
X_val_wv_reg = sp.csr_matrix(pd.concat([val_wv, X_val_reg_df], axis=1))

In [28]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.1,
    max_depth=2, n_estimators=2000, verbose=1
)

xg_cls.fit(X_train_wv_reg, train_df_processed['recommended'])
train_AUC = roc_auc_score(
    train_df_processed['recommended'], xg_cls.predict(X_train_wv_reg))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], xg_cls.predict(X_val_wv_reg))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

/home/farid/miniconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:32:13] WARNING: ../src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:32:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training AUC: 0.6742503129255473
Validation AUC: 0.6137549462562668


In [30]:
clf = LogisticRegression(max_iter = 1000)
clf.fit(X_train_wv_reg, train_df_processed['recommended'])

train_AUC = roc_auc_score(
    train_df_processed['recommended'], clf.predict(X_train_wv_reg))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], clf.predict(X_val_wv_reg))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.653079639219785
Validation AUC: 0.5958531431135363
